In [133]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split

df = pd.read_csv("cleaned_dataset.csv")
#We look at our dataframe 

df.head()
df.drop('Unnamed: 0', axis=1, inplace=True)

Low outliers? Remove, if they represent more than 10% of dataset... Then you cannot just delete it. 

Observations: 
- We see that there are 7338 misisng values for surface area of the land. Those missing values are probably from apartments. When we check the entries of apartments we see exactly 7338 entries. 
- There are a lot of unique values for the columns Garden and Terrace, and this column is also highly dependent on being a house or an apartment. 

Those observations suggest that it's better to split the dataframe in houses and apartments and to make several price predictions. Before doing the split, it's good to do some feature engineering which will have an affect on both property types. 

We will add region Brussel and 10 other provinces as a column 'provinces' according to their postal_code. Furthermore, we'll add an extra column USA_kitchen which will be 1 if there's any kitchen of a USA-style (whether it's fully equipped or not). Our previous datashet showed us that this could have an affect in our sale price so we would like to include this in our model.

In [134]:
df.head()

,Province,Type_of_property,Subtype_of_property,Price,Number_of_rooms,Living_Area,Fully_equipped_kitchen,Furnished,Open_fire,Terrace,Terrace_Area,Garden,Garden_Area,Surface_area_of_the_plot_of_land,Number_of_facades,Swimming_pool,State_of_the_building
0,Brussels,APARTMENT,PENTHOUSE,450000,3,156.0,USA_HYPER_EQUIPPED,False,False,True,43.0,NaN,NaN,NaN,4.0,NaN,NaN
1,Brussels,APARTMENT,PENTHOUSE,450000,3,156.0,USA_HYPER_EQUIPPED,False,False,True,43.0,NaN,NaN,NaN,4.0,NaN,NaN
2,Brussels,APARTMENT,APARTMENT,485000,2,107.0,USA_HYPER_EQUIPPED,False,False,True,17.0,NaN,NaN,NaN,NaN,False,AS_NEW
3,Brussels,APARTMENT,DUPLEX,460000,3,147.0,USA_HYPER_EQUIPPED,False,False,True,52.0,NaN,NaN,NaN,2.0,NaN,NaN
4,Walloon Brabant,APARTMENT,APARTMENT,475000,3,115.0,NOT_INSTALLED,False,False,True,20.0,NaN,NaN,NaN,NaN,NaN,AS_NEW


In [135]:
df.shape

(15005, 17)

In [136]:
df_ap=df[df.Type_of_property=='APARTMENT']
df_house=df[df.Type_of_property=='HOUSE']

We start with apartments

In [137]:
df_ap.describe(include=object)

,Province,Type_of_property,Subtype_of_property,Fully_equipped_kitchen,Furnished,Terrace,Garden,Swimming_pool,State_of_the_building
count,6781,6781,6781,4847,5097,4871,1046,3920,4874
unique,11,1,9,8,2,1,1,2,6
top,West Flanders,APARTMENT,APARTMENT,INSTALLED,False,True,True,False,AS_NEW
freq,1565,6781,5169,2276,4821,4871,1046,3869,2311


In [138]:
df_ap.drop(columns=['Type_of_property','Swimming_pool', 'Surface_area_of_the_plot_of_land'], axis=1, inplace=True)

C:\Users\nacit\AppData\Local\Temp\ipykernel_18328\4097696308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ap.drop(columns=['Type_of_property','Swimming_pool', 'Surface_area_of_the_plot_of_land'], axis=1, inplace=True)


In [139]:
df_ap.head()

,Province,Subtype_of_property,Price,Number_of_rooms,Living_Area,Fully_equipped_kitchen,Furnished,Open_fire,Terrace,Terrace_Area,Garden,Garden_Area,Number_of_facades,State_of_the_building
0,Brussels,PENTHOUSE,450000,3,156.0,USA_HYPER_EQUIPPED,False,False,True,43.0,NaN,NaN,4.0,NaN
1,Brussels,PENTHOUSE,450000,3,156.0,USA_HYPER_EQUIPPED,False,False,True,43.0,NaN,NaN,4.0,NaN
2,Brussels,APARTMENT,485000,2,107.0,USA_HYPER_EQUIPPED,False,False,True,17.0,NaN,NaN,NaN,AS_NEW
3,Brussels,DUPLEX,460000,3,147.0,USA_HYPER_EQUIPPED,False,False,True,52.0,NaN,NaN,2.0,NaN
4,Walloon Brabant,APARTMENT,475000,3,115.0,NOT_INSTALLED,False,False,True,20.0,NaN,NaN,NaN,AS_NEW


In [140]:
corr=df_ap.corr()
corr.Price 



C:\Users\nacit\AppData\Local\Temp\ipykernel_18328\1049142469.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr=df_ap.corr()


Price                1.000000
Number_of_rooms      0.419843
Living_Area          0.175046
Open_fire            0.096877
Terrace_Area         0.260947
Garden_Area         -0.094942
Number_of_facades    0.026987
Name: Price, dtype: float64

According to our correlations we will keep the following columns: Number_of_rooms, Living_Area, Terrace_Area. We keep the values which have a high correlation. For the categorical values, we can't calculate the correlation. We will try another approach. 
To determine the correlation between a categorical variable (such as Province) and a numerical variable (such as Price) using a technique called chi-squared test for independence

In [141]:
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(df_ap['Province'], df_ap['Price'])

# Calculate the chi-squared test for independence
stat, p, dof, expected = chi2_contingency(contingency_table, correction=False)

#print p-value
print(p)

2.2858065600809354e-140


The p-value is very small (<0.05) and says that the probability of this distribution in case of no association between the two variables is almost 0. This is hence a feature we would like to keep. Let's do the same for 

In [142]:
# Create a contingency table
contingency_table3 = pd.crosstab(df_ap['Fully_equipped_kitchen'], df_ap['Price'])

# Calculate the chi-squared test for independence
stat3, p3, dof3, expected3 = chi2_contingency(contingency_table3, correction=False)

#print p-value
print(p3)

0.047686502540815606


We can say that this is significant too and hence we'll use this feature in our dataset. 

Lastly we'll have a look on subtype of property: 


In [143]:
# Create a contingency table
contingency_table4 = pd.crosstab(df_ap['Subtype_of_property'], df_ap['Price'])

# Calculate the chi-squared test for independence
stat4, p4, dof4, expected4 = chi2_contingency(contingency_table4, correction=False)

#print p-value
print(p4)

1.0836869765712786e-28


PART 2: PREPROCESSING

In [144]:
columns_to_keep = ['Price','Subtype_of_property','Province','Number_of_rooms','Terrace_Area','Living_Area','Fully_equipped_kitchen']
df_ap=df_ap[columns_to_keep]

In [145]:
df_ap.head()

,Price,Subtype_of_property,Province,Number_of_rooms,Terrace_Area,Living_Area,Fully_equipped_kitchen
0,450000,PENTHOUSE,Brussels,3,43.0,156.0,USA_HYPER_EQUIPPED
1,450000,PENTHOUSE,Brussels,3,43.0,156.0,USA_HYPER_EQUIPPED
2,485000,APARTMENT,Brussels,2,17.0,107.0,USA_HYPER_EQUIPPED
3,460000,DUPLEX,Brussels,3,52.0,147.0,USA_HYPER_EQUIPPED
4,475000,APARTMENT,Walloon Brabant,3,20.0,115.0,NOT_INSTALLED


In [146]:
df_ap.isnull().sum()

Price                        0
Subtype_of_property          0
Province                     0
Number_of_rooms              0
Terrace_Area              3046
Living_Area                335
Fully_equipped_kitchen    1934
dtype: int64

In [13]:
#df_ap.to_csv("test.csv")

We first want to deal with the missing values for Living area, fully equipped kitchen and terrace area. 

In [44]:
df_ap.Number_of_rooms.value_counts()

2     3538
1     1437
3     1426
4      159
0      155
5       38
6       16
7        3
8        3
10       3
9        2
15       1
Name: Number_of_rooms, dtype: int64

We consider only ap till 6 bedrooms.   (ENTER IN LIMITATION) WRITE A FUNCTION TO DO THINGS IN AN EFFICIENT WAY

In [147]:
df_ap=df_ap[df.Number_of_rooms<7]

C:\Users\nacit\AppData\Local\Temp\ipykernel_18328\3437916107.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_ap=df_ap[df.Number_of_rooms<7]


In [148]:
df_ap.isnull().sum()

Price                        0
Subtype_of_property          0
Province                     0
Number_of_rooms              0
Terrace_Area              3037
Living_Area                333
Fully_equipped_kitchen    1927
dtype: int64

In [65]:
df_ap.isnull().sum()

Price                        0
Subtype_of_property          0
Province                     0
Number_of_rooms              0
Terrace_Area              3037
Living_Area                333
Fully_equipped_kitchen    1927
dtype: int64

In [149]:
# fill missing values in Living_Area column with value depending on BEDROOMS column

df_ap['Living_Area'] = df_ap['Living_Area'].fillna(df_ap.groupby('Number_of_rooms')['Living_Area'].transform('mean'))



In [150]:
df_ap['Living_Area']=df_ap['Living_Area'].astype(int)

In [151]:
df_ap.isnull().sum()

Price                        0
Subtype_of_property          0
Province                     0
Number_of_rooms              0
Terrace_Area              3037
Living_Area                  0
Fully_equipped_kitchen    1927
dtype: int64

In [71]:
df_ap.dtypes

Price                       int64
Subtype_of_property        object
Province                   object
Number_of_rooms             int64
Terrace_Area              float64
Living_Area                 int32
Fully_equipped_kitchen     object
dtype: object

In [72]:
df_ap.to_csv("test2.csv")

#controle OK

When making such changes it's always advises to check your values back. I had some problems before converting to integers.

In [34]:
#df_ap.groupby('Number_of_rooms').mean()

C:\Users\nacit\AppData\Local\Temp\ipykernel_18328\2078779259.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ap.groupby('Number_of_rooms').mean()


,Price,Terrace_Area,Living_Area
Number_of_rooms,,,
0,216278.496774,10.400000,57.328767
1,233110.444676,13.367164,66.471408
2,331931.755229,18.033898,104.002081
3,444113.645161,25.239560,140.143800
4,487118.408805,30.891566,190.039474
5,533973.684211,41.583333,260.027027
6,563231.250000,80.000000,262.272727


For the living area we suppose there is no terrace so we change the value by 0. 

In [152]:
df_ap['Terrace_Area'].fillna(0, inplace=True)
df_ap['Terrace_Area']=df_ap['Terrace_Area'].astype(int)

In [186]:
df_ap.isnull().sum()

Price                     0
Subtype_of_property       0
Province                  0
Number_of_rooms           0
Terrace_Area              0
Living_Area               0
Fully_equipped_kitchen    0
dtype: int64

In [187]:
df_ap.Fully_equipped_kitchen.value_counts()

INSTALLED             2274
0                     1927
HYPER_EQUIPPED        1149
USA_HYPER_EQUIPPED     623
SEMI_EQUIPPED          370
NOT_INSTALLED          198
USA_INSTALLED          180
USA_SEMI_EQUIPPED       44
USA_UNINSTALLED          4
Name: Fully_equipped_kitchen, dtype: int64

We see there are not many USA_type kitchens. We can add them together to categorize it better: 



In [188]:
df_ap.head()

,Price,Subtype_of_property,Province,Number_of_rooms,Terrace_Area,Living_Area,Fully_equipped_kitchen
0,450000,PENTHOUSE,Brussels,3,43,156,USA_HYPER_EQUIPPED
1,450000,PENTHOUSE,Brussels,3,43,156,USA_HYPER_EQUIPPED
2,485000,APARTMENT,Brussels,2,17,107,USA_HYPER_EQUIPPED
3,460000,DUPLEX,Brussels,3,52,147,USA_HYPER_EQUIPPED
4,475000,APARTMENT,Walloon Brabant,3,20,115,NOT_INSTALLED


In [189]:
df_ap = df_ap.replace({"Fully_equipped_kitchen":{"NOT_INSTALLED": 0, "INSTALLED": 1, "SEMI_EQUIPPED": 2, "HYPER_EQUIPPED": 3, "USA_UNINSTALLED": 0, "USA_INSTALLED": 1, "USA_SEMI_EQUIPPED": 2, "USA_HYPER_EQUIPPED": 3}})

In [191]:
df_ap["Fully_equipped_kitchen"].fillna(0, inplace=True)

In [193]:
# Create a contingency table
contingency_table5 = pd.crosstab(df_ap['Fully_equipped_kitchen'], df_ap['Price'])

# Calculate the chi-squared test for independence
stat5, p5, dof5, expected5 = chi2_contingency(contingency_table5, correction=False)

#print p-value
print(p3)

0.047686502540815606


In [192]:
df_ap.isnull().sum()

Price                     0
Subtype_of_property       0
Province                  0
Number_of_rooms           0
Terrace_Area              0
Living_Area               0
Fully_equipped_kitchen    0
dtype: int64

PART 3: MODEL SELECTION? 

In [195]:
X=df_ap.drop('Price', axis=1)
y=df_ap['Price']

In [196]:
# Get the columns name
columns_name = ['Subtype_of_property','Province']
# Label encode them
X = pd.get_dummies(X, columns=columns_name, prefix=columns_name, drop_first=False)

In [197]:
X.head()

,Number_of_rooms,Terrace_Area,Living_Area,Fully_equipped_kitchen,Subtype_of_property_APARTMENT,Subtype_of_property_DUPLEX,Subtype_of_property_FLAT_STUDIO,Subtype_of_property_GROUND_FLOOR,Subtype_of_property_KOT,Subtype_of_property_LOFT,...,Province_Brussels,Province_East Flanders,Province_Flemish Brabant,Province_Hainaut,Province_Limburg,Province_Liège,Province_Luxembourg,Province_Namur,Province_Walloon Brabant,Province_West Flanders
0,3,43,156,3,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,3,43,156,3,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2,17,107,3,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,3,52,147,3,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,3,20,115,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [159]:
#X=X.to_numpy()
#y=y.to_numpy

In [198]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [176]:
from sklearn import linear_model

In [177]:
model = linear_model.Ridge(alpha = 9000)
model.fit(X_train, y_train)

Ridge(alpha=9000)

In [181]:
y_test

194      580000
3105     315000
3169     175000
999      265000
3233     255000
          ...  
3410     260000
714      192000
13517    398000
2840     699000
3325     449000
Name: Price, Length: 1354, dtype: int64

In [179]:
model.predict(X_test)

array([362254.92141715, 331200.20618676, 375743.31309081, ...,
       363751.8578353 , 321859.52288866, 350048.29573284])

In [81]:
y.head()

0    450000
1    450000
2    485000
3    460000
4    475000
Name: Price, dtype: int64

In [82]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [89]:
X=df_ap.drop('Price', axis=1)
y=df_ap['Price']

In [113]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [115]:
from sklearn.linear_model import LinearRegression
lin = LinearRegression()
lin.fit(X_train, y_train)

LinearRegression()

In [122]:
from sklearn import preprocessing

In [124]:
X_scaled = preprocessing.scale(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [126]:
from sklearn.preprocessing import PolynomialFeatures

pft = PolynomialFeatures(degree = 2)
X_poly = pft.fit_transform(X_scaled)

In [127]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, X_scaled,test_size = 0.2,random_state = 42)

In [128]:
from sklearn import linear_model
model = linear_model.Ridge(alpha = 300)
# alpha is the regularization parameter(don't get confused by the symbol)
model.fit(X_train, y_train)

Ridge(alpha=300)

In [129]:
predictionTestSet = model.predict(X_test)

-2.4362071443049734e+19

In [130]:
from sklearn.metrics import mean_squared_error
errorTestSet = mean_squared_error(y_test, predictionTestSet)

In [131]:
print(errorTestSet)

0.0009002883628461652


Having or knowing what data limitations is, is very good. Dipe deever and see what the problems are. What are features you wish you had and how you can solve it later?  Another thing is , look only to the variables driving this source?